In [33]:
import csv
import json
import os
import sys
import glog
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
tf.test.i

In [9]:
os.getcwd()

'/polyai-models/intent_detection'

In [38]:
test = pd.read_csv('/polyai-models/polyai-models/200909_test_huawei_wallet_30strat_min3_manyanswers.csv')
train = pd.read_csv('/polyai-models/polyai-models/200909_train_huawei_wallet_60strat_min3_manyanswers.csv')
val = pd.read_csv('/polyai-models/polyai-models/200909_val_huawei_wallet_10strat_min3_manyanswers.csv')

In [39]:
train.shape

(4676, 7)

In [40]:
train.append(val).shape

(5438, 7)

In [35]:
df = pd.read_csv(test)
df.head()

,Unnamed: 0,Unnamed: 0.1,text,intent,intentId,labels,answer
0,0,6997,Why can only one city be added to the all-in-o...,Which cities can a traffic card be used in?,00154140377723915914286ed488ffbf,20,The following urban transportation cards are s...
1,1,3721,Delete the bank card that is paid on the Inter...,Method of removing bank cards from online payment,00154140377750316557286ed488ffbf,102,"To remove a bank card from online payment, per..."
2,2,7352,Can an Eid be used to stay in a hotel?,Which Scenarios Can EIDs Be Used for?,00154140377719915881286ed488ffbf,9,"<div class=\""qa-answer-main\"">eID can be used ..."
3,3,5828,"Why does the message \""Huawei Pay\"" always pop...",The Huawei Pay card swiping page is displayed.,00154140377723915950286ed488ffbf,50,The reason why the Huawei Pay card swiping scr...
4,4,4315,Recharge process of H-coins,Use the H-coin card to recharge the account.,00154140377750316534286ed488ffbf,81,"To recharge your H-coin card, do as follows: 1..."


In [28]:
with tf.gfile.Open(test, "r") as data_file:
    data = np.array(list(csv.reader(data_file))[1:])
    print(data.shape)
    

(2331, 7)


In [36]:
labels= data[:,5]
text = data[:,2]

In [ ]:
    if hparams.data_regime == "full":
        train_file = "train"
    elif hparams.data_regime == "10":
        train_file = "train_10"
    elif hparams.data_regime == "30":
        train_file = "train_30"
    else:
        glog.error(f"Invalid data regime: {hparams.data_regime}")
    train_data = os.path.join(
        data_dir, hparams.task, f"{train_file}.csv")
    test_data = os.path.join(data_dir, hparams.task, "test.csv")
    categories_file = os.path.join(data_dir, hparams.task, "categories.json")

    with tf.gfile.Open(categories_file, "r") as categories_file:
        categories = json.load(categories_file)
    
    
    labels = {}
    encodings = {}

    with tf.gfile.Open(train_data, "r") as data_file:
        data = np.array(list(csv.reader(data_file))[1:])
        labels[_TRAIN] = data[:, 1]
        encodings[_TRAIN] = encoder_client.encode_sentences(data[:, 0])

    with tf.gfile.Open(test_data, "r") as data_file:
        data = np.array(list(csv.reader(data_file))[1:])
        labels[_TEST] = data[:, 1]
        encodings[_TEST] = encoder_client.encode_sentences(data[:, 0])

    # convert labels to integers
    labels = {
        k: np.array(
            [categories.index(x) for x in v]) for k, v in labels.items()
    }

In [6]:
_TRAIN = "train"
_TEST = "test"


def _preprocess_data(encoder_client, hparams, data_dir):
    """Reads the data from the files, encodes it and parses the labels

    Args:
        encoder_client: an EncoderClient
        hparams: a tf.contrib.training.HParams object containing the model
            and training hyperparameters
        data_dir: The directory where the inten data has been downloaded

    Returns:
        categories, encodings, labels

    """
    if hparams.data_regime == "full":
        train_file = "train"
    elif hparams.data_regime == "10":
        train_file = "train_10"
    elif hparams.data_regime == "30":
        train_file = "train_30"
    else:
        glog.error(f"Invalid data regime: {hparams.data_regime}")
    train_data = os.path.join(
        data_dir, hparams.task, f"{train_file}.csv")
    test_data = os.path.join(data_dir, hparams.task, "test.csv")
    categories_file = os.path.join(data_dir, hparams.task, "categories.json")

    with tf.gfile.Open(categories_file, "r") as categories_file:
        categories = json.load(categories_file)

    labels = {}
    encodings = {}

    with tf.gfile.Open(train_data, "r") as data_file:
        data = np.array(list(csv.reader(data_file))[1:])
        labels[_TRAIN] = data[:, 1]
        encodings[_TRAIN] = encoder_client.encode_sentences(data[:, 0])

    with tf.gfile.Open(test_data, "r") as data_file:
        data = np.array(list(csv.reader(data_file))[1:])
        labels[_TEST] = data[:, 1]
        encodings[_TEST] = encoder_client.encode_sentences(data[:, 0])

    # convert labels to integers
    labels = {
        k: np.array(
            [categories.index(x) for x in v]) for k, v in labels.items()
    }

    return categories, encodings, labels


In [2]:
def _main():
    parsed_args, hparams = parse_args_and_hparams()

    if hparams.task.lower() not in ["clinc", "hwu", "banking"]:
        raise ValueError(f"{hparams.task} is not a valid task")

    encoder_client = get_encoder_client(hparams.encoder_type,
                                        cache_dir=hparams.cache_dir)

    categories, encodings, labels = _preprocess_data(
        encoder_client, hparams, parsed_args.data_dir)

    accs = []
    eval_acc_histories = []
    if hparams.eval_each_epoch:
        validation_data = (encodings[_TEST], labels[_TEST])
        verbose = 1
    else:
        validation_data = None
        verbose = 0

    for seed in range(hparams.seeds):
        glog.info(f"### Seed {seed} ###")
        model, eval_acc_history = train_model(
            encodings[_TRAIN], labels[_TRAIN], categories, hparams,
            validation_data=validation_data, verbose=verbose)

        _, acc = model.evaluate(encodings[_TEST], labels[_TEST], verbose=0)
        glog.info(f"Seed accuracy: {acc:.3f}")
        accs.append(acc)
        eval_acc_histories.append(eval_acc_history)

    average_acc = np.mean(accs)
    variance = np.std(accs)
    glog.info(
        f"Average results:\n"
        f"Accuracy: {average_acc:.3f}\n"
        f"Variance: {variance:.3f}")

    results = {
        "Average results": {
            "Accuracy": float(average_acc),
            "Variance": float(variance)
        }
    }
    if hparams.eval_each_epoch:
        results["Results per epoch"] = [
            [float(x) for x in y] for y in eval_acc_histories]

    if not tf.gfile.Exists(parsed_args.output_dir):
        tf.gfile.MakeDirs(parsed_args.output_dir)
    with tf.gfile.Open(
            os.path.join(parsed_args.output_dir, "results.json"), "w") as f:
        json.dump(results, f, indent=2)

In [3]:
def train_model(train_encodings, train_labels, categories, hparams,
                validation_data=None, verbose=1):
    """Trains an intent classification model

    Args:
        train_encodings: np.array with the train encodings
        train_labels: list of labels corresponding to each train example
        categories: the set of categories
        hparams: a tf.contrib.training.HParams object containing the model
            and training hyperparameters
        validation_data: (validation_encodings, validation_labels) tuple
        verbose: the keras_model.train() verbose level

    Returns:
        model: a keras model
        eval_acc_history: The evaluation results per epoch

    """
    distribution = None if not hparams.balance_data else {
        x: 1. / len(categories) for x in range(len(categories))}

    batcher = SamplingBatcher(
        train_encodings, train_labels, hparams.batch_size, distribution)

    steps_per_epoch = np.ceil(len(train_labels) / hparams.batch_size)

    model, eval_acc_history = _train_mlp_with_generator(
        batcher, train_encodings.shape[1], steps_per_epoch,
        categories, hparams, validation_data=validation_data, verbose=verbose)
    return model, eval_acc_history


In [4]:
from collections import abc
from typing import Dict, Optional

import numpy as np

_MAX_PER_BATCH = 3


class SamplingBatcher(abc.Iterator):
    """Batcher that samples according to a given distribution.

    It defaults to sampling from the data distribution.

    WARNING: this class is not deterministic. if you want deterministic
    behaviour, just freeze the numpy seed.
    """
    def __init__(
            self,
            examples: np.ndarray,
            labels: np.ndarray,
            batch_size: int,
            sample_distribution: Optional[Dict[int, float]] = None,
    ):
        """Create a new BalancedBatcher.

        Args:
            examples: np.ndarray containing examples
            labels: np.ndarray containing labels
            batch_size: int size of a single batch
            sample_distribution: optional distribution over label
                classes for sampling. This is normalized to sum to 1. Defines
                the target distribution that batches will be sampled with.
                Defaults to the data distribution.
        """
        _validate_labels_examples(examples, labels)
        self._examples = examples
        self._labels = labels
        self._label_classes = np.unique(labels)
        self._class_to_indices = {
            label: np.argwhere(labels == label).flatten()
            for label in self._label_classes
        }
        if sample_distribution is None:
            # Default to the data distribution
            sample_distribution = {
                label: float(indices.size)
                for label, indices in self._class_to_indices.items()
            }
        self._label_choices, self._label_probs = (
            self._get_label_choices_and_probs(sample_distribution))
        self._batch_size = batch_size

    def _get_label_choices_and_probs(self, sample_distribution):
        label_choices = []
        label_probs = []
        weight_sum = sum(sample_distribution.values())
        for label, weight in sample_distribution.items():
            if label not in self._labels:
                raise ValueError(
                    f"label {label} in sample distribution does not exist")
            if weight < 0.0:
                raise ValueError(
                    f"weight {weight} for label {label} is negative")
            label_choices.append(label)
            label_probs.append(weight / weight_sum)

        return np.array(label_choices), np.array(label_probs)

    def __next__(self):
        """Generates the next batch.

        Returns:
            (batch_of_examples, batch_of_labels) - a tuple of ndarrays
        """
        class_choices = np.random.choice(
            self._label_choices, size=self._batch_size, p=self._label_probs)

        batch_indices = []
        for class_choice in class_choices:
            indices = self._class_to_indices[class_choice]
            batch_indices.append(np.random.choice(indices))

        return self._examples[batch_indices], self._labels[batch_indices]

    def __iter__(self):
        """Gets an iterator for this iterable

        Returns:
            self because the class is an iterator itself
        """
        return self